<a id='top'></a>

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# %matplotlib nbagg 
%matplotlib notebook
# %matplotlib inline



## Third party 
import numpy as np
import os, time, zarr, sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import unslice.IO as io
from unslice.utils import *
from unslice.registration.featmatch import *
from unslice.registration.transform import *
from unslice.registration.rigid import *
from unslice.registration.gpu_transform import *
from unslice.registration.utils import *
from unslice.segmentation import *
from unslice.tracing.pyoof import OOF, apply_oof_v2
from unslice.tracing.skel import *
from unslice.flatten import *
from unslice.lightsheetcorrect import *




In [2]:
# Parameters that are constant throughout notebook
working_dir = '/mnt/share3/webster/MORF8_3and4/MORF8_3and4'

def bdir(fname):
    return os.path.join(working_dir, fname)

# prefix to add to the beginning of each filename 
name_prefix = 'L-v5' #left
name_prefix2 = 'R-v5' #right

# Table of contents

### Pre-processing
[1. Convert to zarr](#convert)<br>

### Warp using lectin deformation fields 
[13. Flatten warp](#flattenwarp)<br>
[14. TPS lectin warp](#anchorwarp)<br>

### Transformation (round 1)
[16. Feature match](#featmatch)<br>
[17. TPS transformation (Round 1)](#tpswarp)<br>

[Warp points back, use all at once](#theend)<br>
[Round 2 warp](#round2)

# Convert to zarr
<a id='convert'></a>

[Return to top](#top)

In [3]:
# Parameters for converting to zarr

slab_path = '/mnt/share3/webster/MORF8_3and4/left_V5_tiffs_corrected'
slab_zarr_path = bdir(name_prefix+'.zarr')
pc2_img_size = (3900,3400,400)


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (1,231 x 1,74 x 1,231)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
zrange = None
xrange = None
yrange = None

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:0-3900, y:0-3400, z:0-40


100%|██████████| 40/40 [00:17<00:00,  2.30it/s]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 340/340 [00:22<00:00, 15.34it/s]

Processing chunk x:0-3900, y:0-3400, z:40-80


...

Processing chunk x:0-3900, y:0-3400, z:360-400


100%|██████████| 40/40 [00:16<00:00,  2.48it/s]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 340/340 [01:08<00:00,  4.93it/s]


742.4068751335144 seconds elapsed


In [4]:
# Parameters for converting to zarr

# Right

slab_path = '/mnt/share3/webster/MORF8_3and4/right_V5_tiffs_corrected'
slab_zarr_path = bdir(name_prefix2+'.zarr')
pc2_img_size = (3900,3250,360)


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (1,231 x 1,74 x 1,231)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
zrange = None
xrange = None
yrange = None

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:0-3900, y:0-3250, z:0-40


100%|██████████| 40/40 [00:15<00:00,  2.56it/s]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 340/340 [00:26<00:00, 13.02it/s]


Processing chunk x:0-3900, y:0-3250, z:40-80


...

Processing chunk x:0-3900, y:0-3250, z:320-360



100%|██████████| 40/40 [00:11<00:00,  3.39it/s]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 340/340 [00:32<00:00, 10.46it/s]


426.5372402667999 seconds elapsed


## Flatten warp the slabs

[Return to top](#top)

<a id='flattenwarp'></a>

In [7]:
fixed_zarr_path = bdir(name_prefix+'.zarr') 
moving_zarr_path = bdir(name_prefix+'.zarr')
warped_zarr_path = bdir(name_prefix+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = (32,32,32)
chunks=3*(200,)
nb_workers = 8 


# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('warping_grids/L-lec_grid_flatten.npy')


##########################

moving_pts_paths = [bdir('L-lec_downsampled_thinsurface_top_endpts_contourfiltered_outlier_new.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('L-lec_downsampled_thinsurface_top_endpts_contourfiltered_outlier_new.npy')]

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(3900, 3400, 400)
Loading grid values...
Warping image...
Moving image size: 10.608 GB


100%|██████████| 680/680 [01:21<00:00,  8.36it/s]


Time elapsed: 2.672732 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:51<00:00,  3.92it/s]


Loading z 200 - 400


100%|██████████| 200/200 [00:50<00:00,  3.94it/s]


In [8]:
fixed_zarr_path = bdir(name_prefix2+'.zarr') 
moving_zarr_path = bdir(name_prefix2+'.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = (32,32,32)
chunks=3*(200,)
nb_workers = 8 


# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('warping_grids/R-lec_grid_flatten.npy')


##########################

moving_pts_paths = [bdir('L-lec_downsampled_thinsurface_top_endpts_contourfiltered_outlier_new.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('L-lec_downsampled_thinsurface_top_endpts_contourfiltered_outlier_new.npy')]

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
# tiff_path = warped_zarr_path[:-5]+'_tiffs'
# convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(3900, 3250, 360)
Loading grid values...
Warping image...
Moving image size: 9.126 GB


100%|██████████| 680/680 [01:25<00:00,  8.00it/s]


Time elapsed: 2.742539 minutes


## TPS warp using lectin grid

[Return to top](#top)

<a id='anchorwarp'></a>

In [10]:
# Top slab 
fixed_zarr_path = bdir('R-lec_flattened_anchorwarp_r3.zarr') # We have to make this the fixed zarr so that it'll warp to the correct shape 
moving_zarr_path = bdir(name_prefix2+'_flattened.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_lectinwarp.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(32,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('warping_grids/grid_tps_r3.npy')

# Number of z slices to add to the final image 
zadd = 564-360 


##########################

moving_pts_paths = [bdir('L-lec_downsampled_thinsurface_top_endpts_contourfiltered_outlier_new.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('L-lec_downsampled_thinsurface_top_endpts_contourfiltered_outlier_new.npy')]

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None


TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path, zadd=zadd, 
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(3900, 3400, 564)
Loading grid values...
Warping image...
Moving image size: 9.126 GB


100%|██████████| 1020/1020 [01:29<00:00, 11.44it/s]


Time elapsed: 2.596741 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:50<00:00,  3.93it/s]


Loading z 200 - 400


100%|██████████| 200/200 [00:50<00:00,  3.95it/s]


Loading z 400 - 564


100%|██████████| 164/164 [00:41<00:00,  3.99it/s]
